In [58]:
import pandas as pd

In [59]:
dfs = pd.read_excel('operator.xlsx', sheet_name=None, usecols=[0, 1, 3, 5, 7], skipcols=[2, 4, 6])
dfs

OrderedDict([('1000',                     A環境 Unnamed: 1 Unnamed: 3 Unnamed: 5
              0                   NaN        NaN        NaN        NaN
              1                    日付        責任者          A          B
              2   2019-10-12 00:00:00      東京　太郎       hoge      tokyo
              3   2019-10-13 00:00:00      大阪　花子      東京　太郎      osaka
              4   2019-10-14 00:00:00       jiro        bar     nagoya
              5   2019-10-15 00:00:00     ichiro       hoge      tokyo
              6   2019-10-16 00:00:00     hanako        foo      osaka
              7   2019-10-17 00:00:00       taro        bar     nagoya
              8   2019-10-18 00:00:00       jiro       hoge      tokyo
              9   2019-10-19 00:00:00        NaN        NaN        NaN
              10  2019-10-20 00:00:00        NaN        NaN        NaN
              11  2019-10-21 00:00:00     hanako       hoge      tokyo
              12  2019-10-22 00:00:00       taro        foo      osak

In [60]:
to_rocketchat_user = {
    '東京　太郎': 'tokyotaro',
    '大阪　花子': 'osakahanako'
}   

In [61]:
json_dict = {}
for _, value in dfs.items():
    df = value

    # テスト環境の取得
    env_name = df.columns.values[0]
    # 4行目以降のループ
    for _, columns in df[2:].iterrows(): 
        # 日付をYYYY-MM-DD形式の文字列に変換
        date = columns[0].strftime("%Y-%m-%d")
        
        # 日付をキーに設定
        if date not in json_dict:
            json_dict[date] = {}
        
        # オペレータの名前をリスト化
        operators = []
        leader = None
        for index, name in enumerate(columns[1:]):
            # 空のセルは無視
            if type(name) is not type(pd.NaT):
                if index == 0:
                    if  name in to_rocketchat_user:
                        leader = name + '( @' + to_rocketchat_user[name] + ' )'
                    else:
                        leader = name
                else:
                    if  name in to_rocketchat_user:
                        operators.append(name + '( @' + to_rocketchat_user[name] + ' )' )
                    else:             
                        operators.append(name)
            
        json_dict[date][env_name] = {
            '責任者': leader,
            'オペレーター':operators
        }

In [62]:
json_dict

{'2019-10-12': {'A環境': {'責任者': '東京\u3000太郎( @tokyotaro )',
   'オペレーター': ['hoge', 'tokyo']},
  'B環境': {'責任者': 'hanako', 'オペレーター': ['hoge', 'tokyo']}},
 '2019-10-13': {'A環境': {'責任者': '大阪\u3000花子( @osakahanako )',
   'オペレーター': ['東京\u3000太郎( @tokyotaro )', 'osaka']},
  'B環境': {'責任者': 'taro', 'オペレーター': ['foo', 'osaka']}},
 '2019-10-14': {'A環境': {'責任者': 'jiro', 'オペレーター': ['bar', 'nagoya']},
  'B環境': {'責任者': 'jiro', 'オペレーター': ['bar', 'nagoya']}},
 '2019-10-15': {'A環境': {'責任者': 'ichiro', 'オペレーター': ['hoge', 'tokyo']},
  'B環境': {'責任者': 'ichiro', 'オペレーター': ['hoge', 'tokyo']}},
 '2019-10-16': {'A環境': {'責任者': 'hanako', 'オペレーター': ['foo', 'osaka']},
  'B環境': {'責任者': 'hanako', 'オペレーター': ['foo', 'osaka']}},
 '2019-10-17': {'A環境': {'責任者': 'taro', 'オペレーター': ['bar', 'nagoya']},
  'B環境': {'責任者': 'taro', 'オペレーター': ['bar', 'nagoya']}},
 '2019-10-18': {'A環境': {'責任者': 'jiro', 'オペレーター': ['hoge', 'tokyo']},
  'B環境': {'責任者': 'jiro', 'オペレーター': ['hoge', 'tokyo']}},
 '2019-10-19': {'A環境': {'責任者': None, 'オペレーター': []}

In [63]:
import json
with open('result.json', 'w') as outfile:
    json.dump(json_dict, outfile, indent=2, ensure_ascii=False)